In [307]:
import pygsti
from pygsti.circuits import Circuit
import numpy as np

$$
    R_{ij} = \frac{1}{d} \text{Tr}[ \sigma_i E(\sigma_j)]
$$

$$
    p_{ij} = \text{Tr}[\Lambda_E (\rho^T \otimes \Pi_j)]
$$

In [308]:
def PauliMatrix(i):
    if i == 0:
        return np.array([[1, 0], [0, 1]])
    elif i == 1:
        return np.array([[0, 1], [1, 0]])
    elif i == 2:
        return np.array([[0, -1j], [1j, 0]])
    elif i == 3:
        return np.array([[1, 0], [0, -1]])
    else:
        raise ValueError("i must be 0, 1, 2, or 3.")

def PauliTensor(i, j):
    return np.kron(PauliMatrix(i), PauliMatrix(j))

In [309]:
class ProcessTomographyExperiment_2Q:
    def __init__(self, qubit_labels):
        self.qubit_labels = qubit_labels
        
    def all_prep_fiducials(self):
        return {'0' : '', 
                '1' : 'xx',
                '+' : 'y', 
                '-' : 'yyy', 
                'i' : 'xxx', 
                '-i' : 'x'}
    
    def all_meas_fiducials(self):
        return {'0' : '', 
                '1' : 'xx',
                '+' : 'y', 
                '-' : 'yyy', 
                'i' : 'xxx', 
                '-i' : 'x'}

$$
    p_n = \langle \langle \rho^T_n \otimes \Pi_n | \Lambda \rangle \rangle 
$$

Find a set $\{\langle \langle \rho^T_n \otimes \Pi_n |\}_n$ that spans $L(\mathcal{X} \otimes \mathcal{X})$

https://github.com/qiskit-community/qiskit-community-tutorials/blob/master/ignis/tomography-overview.ipynb

In [310]:
full_pauli_basis = {
    '0' : (np.eye(2) + PauliMatrix(3))/2,
    '1' : (np.eye(2) - PauliMatrix(3))/2,
    '+' : (np.eye(2) + PauliMatrix(1))/2, 
    '-' : (np.eye(2) - PauliMatrix(1))/2,
    'r' : (np.eye(2) + PauliMatrix(2))/2,
    'l' : (np.eye(2) - PauliMatrix(2))/2
}

In [372]:
def make_all_fiducials():
    all_fiducials = {}
    for prep_fiducial in ['0', '1', '+', '-', 'r', 'l']:
        for meas_fiducial in ['0', '1', '+', '-', 'r', 'l']:
            all_fiducials[prep_fiducial+'.'+meas_fiducial] = 2*np.kron(full_pauli_basis[prep_fiducial].T, full_pauli_basis[meas_fiducial])
    return all_fiducials

In [373]:
all_fids = make_all_fiducials()
len(all_fids)

36

In [374]:
from pygsti.modelpacks import smq1Q_XYI
from pygsti.modelmembers.operations import create_from_unitary_mx
from scipy.linalg import expm, logm
from pygsti.tools.jamiolkowski import jamiolkowski_iso
from pygsti.tools.basistools import change_basis
from pygsti.modelmembers.operations import FullArbitraryOp

model = smq1Q_XYI.target_model('static')
# make a random unitary with some depolarization

depol_strength = 0.0
def random_unitary():
    phases = np.random.rand(3)*2*np.pi
    U = expm(-(1j/2)*(phases[0]*PauliMatrix(1) + phases[1]*PauliMatrix(2) + phases[2]*PauliMatrix(3)))
    return U

U = random_unitary()
Gu = create_from_unitary_mx(U, 'full', 'std').to_dense()
Gu = change_basis(Gu, 'std', 'pp')
model['Gu'] = FullArbitraryOp(Gu, 'pp', 'densitymx')
model['Gu'].depolarize(depol_strength)

In [375]:
model['Gu'].to_dense()

array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.99917673,  0.01558151, -0.03745758],
       [ 0.        , -0.00899611,  0.98541342,  0.16993956],
       [ 0.        ,  0.03955912, -0.16946268,  0.98474234]])

In [376]:
choi_Gu = jamiolkowski_iso(model['Gu'].to_dense(), 'pp')

In [377]:
# make design matrix 
# https://github.com/qiskit-community/qiskit-community-tutorials/blob/master/ignis/tomography-overview.ipynb
all_fids = make_all_fiducials()
design_matrix = np.zeros((36, 4**2), dtype=complex)
for idx, fid in enumerate(all_fids.values()):
    design_matrix[idx, :] = fid.flatten()

In [378]:
design_matrix.shape

(36, 16)

In [379]:
def prep_name_to_gates(prep_name):
    if prep_name == '0':
        return []
    elif prep_name == '1':
        return [('Gxpi2', 0), ('Gxpi2', 0)]
    elif prep_name == '+':
        return [('Gypi2', 0)]
    elif prep_name == '-':
        return [('Gypi2', 0), ('Gypi2', 0), ('Gypi2', 0)]
    elif prep_name == 'r':
        return [('Gxpi2', 0), ('Gxpi2', 0),  ('Gxpi2', 0)]
    elif prep_name == 'l':
        return [('Gxpi2', 0)]
    else:
        raise ValueError("prep_name must be 0, 1, +, -, r, or l.")

def meas_name_to_gates(meas_name):
    if meas_name == '0':
        return []
    elif meas_name == '+':
        return [('Gypi2', 0), ('Gypi2', 0), ('Gypi2', 0)]
    elif meas_name == 'r':
        return [('Gxpi2', 0)]
    else:
        raise ValueError("prep_name must be 0, 1, +, -, r, or l.")


def make_circuit_map(fiducial_dict):
    circuit_map = {}
    for prep in ['0', '1', '+', '-', 'r', 'l']:
        for meas in ['0', '+', 'r']:
            key = prep + '.' + meas
            prep_gates = prep_name_to_gates(prep)
            meas_gates = meas_name_to_gates(meas)
            circuit_map[key] = Circuit(prep_gates + ['Gu'] + meas_gates)
    return circuit_map

In [380]:
# make the circuits 
edesign = make_circuit_map(all_fids)
len(edesign)

18

In [381]:
edesign

{'0.0': Circuit(Gu),
 '0.+': Circuit(GuGypi2:0Gypi2:0Gypi2:0@(0)),
 '0.r': Circuit(GuGxpi2:0@(0)),
 '1.0': Circuit(Gxpi2:0Gxpi2:0Gu@(0)),
 '1.+': Circuit(Gxpi2:0Gxpi2:0GuGypi2:0Gypi2:0Gypi2:0@(0)),
 '1.r': Circuit(Gxpi2:0Gxpi2:0GuGxpi2:0@(0)),
 '+.0': Circuit(Gypi2:0Gu@(0)),
 '+.+': Circuit(Gypi2:0GuGypi2:0Gypi2:0Gypi2:0@(0)),
 '+.r': Circuit(Gypi2:0GuGxpi2:0@(0)),
 '-.0': Circuit(Gypi2:0Gypi2:0Gypi2:0Gu@(0)),
 '-.+': Circuit(Gypi2:0Gypi2:0Gypi2:0GuGypi2:0Gypi2:0Gypi2:0@(0)),
 '-.r': Circuit(Gypi2:0Gypi2:0Gypi2:0GuGxpi2:0@(0)),
 'r.0': Circuit(Gxpi2:0Gxpi2:0Gxpi2:0Gu@(0)),
 'r.+': Circuit(Gxpi2:0Gxpi2:0Gxpi2:0GuGypi2:0Gypi2:0Gypi2:0@(0)),
 'r.r': Circuit(Gxpi2:0Gxpi2:0Gxpi2:0GuGxpi2:0@(0)),
 'l.0': Circuit(Gxpi2:0Gu@(0)),
 'l.+': Circuit(Gxpi2:0GuGypi2:0Gypi2:0Gypi2:0@(0)),
 'l.r': Circuit(Gxpi2:0GuGxpi2:0@(0))}

In [382]:
def simulate_data(model, edesign, n_samples=1000):
    data = {}
    for key, circuit in edesign.items():
        prep, meas = key.split('.')
        if meas == '0':
            p0 = model.probabilities(circuit)['0']
            p0 = np.clip(p0, 0, 1)
            counts0 = np.random.binomial(n_samples, p0)
            counts1 = n_samples - counts0
            data[prep+'.0'] = counts0/n_samples
            data[prep+'.1'] = counts1/n_samples
        elif meas == '+':
            p0 = model.probabilities(circuit)['0']
            p0 = np.clip(p0, 0, 1)
            counts0 = np.random.binomial(n_samples, p0)
            counts1 = n_samples - counts0
            data[prep+'.+'] = counts0/n_samples
            data[prep+'.-'] = counts1/n_samples
        elif meas == 'r':
            p0 = model.probabilities(circuit)['0']
            p0 = np.clip(p0, 0, 1)
            counts0 = np.random.binomial(n_samples, p0)
            counts1 = n_samples - counts0
            data[prep+'.r'] = counts0/n_samples
            data[prep+'.l'] = counts1/n_samples
    return data

In [383]:
data = simulate_data(model, edesign, n_samples=10**10)

In [384]:
data_vector = np.array([data[key] for key in sorted(data.keys())]).flatten()

In [385]:
print(design_matrix.shape, data_vector.shape)

(36, 16) (36,)


In [386]:
rho = np.linalg.lstsq(design_matrix, data_vector, rcond=None)[0]

In [387]:
choi_est = np.reshape(rho, (4, 4))

In [388]:
choi_est

array([[ 4.99793858e-01+2.41106354e-18j,  9.89248977e-03+2.24567846e-03j,
        -9.36395640e-03+3.89500448e-03j,  4.92539079e-01-8.48528618e-02j],
       [ 9.89248977e-03-2.24567846e-03j,  2.06142175e-04+2.03830008e-17j,
        -1.67662650e-04+1.17605825e-04j,  9.36395640e-03-3.89500448e-03j],
       [-9.36395640e-03-3.89500447e-03j, -1.67662650e-04-1.17605825e-04j,
         2.05590475e-04-2.58040117e-17j, -9.89144213e-03-2.24774119e-03j],
       [ 4.92539079e-01+8.48528618e-02j,  9.36395640e-03+3.89500447e-03j,
        -9.89144213e-03+2.24774119e-03j,  4.99794410e-01+2.92734587e-18j]])

In [389]:
np.trace(choi_est)

(0.9999999999999992-8.260146078438539e-20j)

In [390]:
choi_Gu

array([[9.92333125e-01+0.j        , 0.00000000e+00-0.08485056j,
        0.00000000e+00-0.01925418j, 0.00000000e+00-0.0061444j ],
       [0.00000000e+00+0.08485056j, 7.25524226e-03+0.j        ,
        1.64634995e-03+0.j        , 5.25384198e-04+0.j        ],
       [0.00000000e+00+0.01925418j, 1.64634995e-03+0.j        ,
        3.73587546e-04+0.j        , 1.19219485e-04+0.j        ],
       [0.00000000e+00+0.0061444j , 5.25384198e-04+0.j        ,
        1.19219485e-04+0.j        , 3.80453947e-05+0.j        ]])

In [391]:
np.trace(choi_Gu)

(1+0j)

In [329]:
from scipy.linalg import sqrtm

In [330]:
np.trace(sqrtm(np.conj(choi_est - choi_Gu).T @ (choi_est - choi_Gu)))

(2.3135069687484737522-1.4550164755645648577e-13j)